<a href="https://colab.research.google.com/github/saroj-shah/NLP_Gen_AI/blob/main/nlp_IMDB_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pwd

'/content'

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.datasets import imdb

In [5]:
# Step 1: Load IMDB Dataset
max_features = 10000  # Only consider the top 10,000 words
max_len = 200  # Pad sequences to a maximum length
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [23]:
# Understanding the dataset
print("Training data shape:", x_train.shape)  # Shape of training data
print("Test data shape:", x_test.shape)      # Shape of test data
print("First training sample:", x_train[0])  # First review (tokenized as integers)
print("First label:", y_train[0])           # Corresponding label (0 = negative, 1 = positive)

Training data shape: (25000, 200)
Test data shape: (25000, 200)
First training sample: [   5   25  100   43  838  112   50  670    2    9   35  480  284    5
  150    4  172  112  167    2  336  385   39    4  172 4536 1111   17
  546   38   13  447    4  192   50   16    6  147 2025   19   14   22
    4 1920 4613  469    4   22   71   87   12   16   43  530   38   76
   15   13 1247    4   22   17  515   17   12   16  626   18    2    5
   62  386   12    8  316    8  106    5    4 2223 5244   16  480   66
 3785   33    4  130   12   16   38  619    5   25  124   51   36  135
   48   25 1415   33    6   22   12  215   28   77   52    5   14  407
   16   82    2    8    4  107  117 5952   15  256    4    2    7 3766
    5  723   36   71   43  530  476   26  400  317   46    7    4    2
 1029   13  104   88    4  381   15  297   98   32 2071   56   26  141
    6  194 7486   18    4  226   22   21  134  476   26  480    5  144
   30 5535   18   51   36   28  224   92   25  104    4  226 

In [6]:
# Decode a review to understand the dataset
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}
def decode_review(text):
    return " ".join([reverse_word_index.get(i - 3, "?") for i in text])

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [24]:
# Decode the first review (convert integers back to words)
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}
decoded_review = " ".join([reverse_word_index.get(i - 3, "?") for i in x_train[0]])  # Offset for reserved indices
print("Decoded review (first sample):", decoded_review)

Decoded review (first sample): and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all t

In [9]:
# Step 2: Preprocessing and Padding
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

In [25]:
# Check the shape after padding
print("Shape after padding (train):", x_train.shape)
print("Shape after padding (test):", x_test.shape)
print("First padded review:", x_train[0])

Shape after padding (train): (25000, 200)
Shape after padding (test): (25000, 200)
First padded review: [   5   25  100   43  838  112   50  670    2    9   35  480  284    5
  150    4  172  112  167    2  336  385   39    4  172 4536 1111   17
  546   38   13  447    4  192   50   16    6  147 2025   19   14   22
    4 1920 4613  469    4   22   71   87   12   16   43  530   38   76
   15   13 1247    4   22   17  515   17   12   16  626   18    2    5
   62  386   12    8  316    8  106    5    4 2223 5244   16  480   66
 3785   33    4  130   12   16   38  619    5   25  124   51   36  135
   48   25 1415   33    6   22   12  215   28   77   52    5   14  407
   16   82    2    8    4  107  117 5952   15  256    4    2    7 3766
    5  723   36   71   43  530  476   26  400  317   46    7    4    2
 1029   13  104   88    4  381   15  297   98   32 2071   56   26  141
    6  194 7486   18    4  226   22   21  134  476   26  480    5  144
   30 5535   18   51   36   28  224   92   2

In [10]:
# Step 3: Build Models with Different Architectures
embedding_dim = 50

In [11]:
def build_rnn_model():
    model = Sequential([
        Embedding(max_features, embedding_dim, input_length=max_len),
        LSTM(64, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    return model

In [12]:
def build_lstm_model():
    model = Sequential([
        Embedding(max_features, embedding_dim, input_length=max_len),
        LSTM(64),
        Dense(1, activation='sigmoid')
    ])
    return model

In [13]:
def build_gru_model():
    model = Sequential([
        Embedding(max_features, embedding_dim, input_length=max_len),
        GRU(64),
        Dense(1, activation='sigmoid')
    ])
    return model

In [14]:
# Step 4: Train and Evaluate Models
def train_and_evaluate(model, x_train, y_train, x_test, y_test):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.2)
    y_pred = (model.predict(x_test) > 0.5).astype("int32")
    acc = accuracy_score(y_test, y_pred)
    return acc, classification_report(y_test, y_pred)

In [15]:
rnn_model = build_rnn_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [16]:
lstm_model = build_lstm_model()

In [17]:
gru_model = build_gru_model()

In [18]:
rnn_acc, rnn_report = train_and_evaluate(rnn_model, x_train, y_train, x_test, y_test)

Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 78s 224ms/step - accuracy: 0.7015 - loss: 0.5480 - val_accuracy: 0.8604 - val_loss: 0.3353
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 65s 170ms/step - accuracy: 0.8981 - loss: 0.2584 - val_accuracy: 0.8740 - val_loss: 0.3103
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 171ms/step - accuracy: 0.9303 - loss: 0.1855 - val_accuracy: 0.8630 - val_loss: 0.3954
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 40ms/step


In [19]:
lstm_acc, lstm_report = train_and_evaluate(lstm_model, x_train, y_train, x_test, y_test)

Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 63s 192ms/step - accuracy: 0.6703 - loss: 0.5640 - val_accuracy: 0.8424 - val_loss: 0.3706
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 75s 170ms/step - accuracy: 0.9008 - loss: 0.2510 - val_accuracy: 0.8694 - val_loss: 0.3074
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 169ms/step - accuracy: 0.9392 - loss: 0.1721 - val_accuracy: 0.8650 - val_loss: 0.3460
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step


In [20]:
gru_acc, gru_report = train_and_evaluate(gru_model, x_train, y_train, x_test, y_test)

Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 66s 203ms/step - accuracy: 0.6418 - loss: 0.6016 - val_accuracy: 0.8498 - val_loss: 0.3503
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 204ms/step - accuracy: 0.8955 - loss: 0.2637 - val_accuracy: 0.8536 - val_loss: 0.3521
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 61s 196ms/step - accuracy: 0.9317 - loss: 0.1849 - val_accuracy: 0.8698 - val_loss: 0.3556
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step


In [21]:
# Compare Performance
print("RNN Accuracy:", rnn_acc)
print(rnn_report)
print("\nLSTM Accuracy:", lstm_acc)
print(lstm_report)
print("\nGRU Accuracy:", gru_acc)
print(gru_report)

RNN Accuracy: 0.85112
              precision    recall  f1-score   support

           0       0.81      0.91      0.86     12500
           1       0.90      0.79      0.84     12500

    accuracy                           0.85     25000
   macro avg       0.86      0.85      0.85     25000
weighted avg       0.86      0.85      0.85     25000


LSTM Accuracy: 0.85832
              precision    recall  f1-score   support

           0       0.82      0.91      0.87     12500
           1       0.90      0.80      0.85     12500

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000


GRU Accuracy: 0.86312
              precision    recall  f1-score   support

           0       0.88      0.84      0.86     12500
           1       0.84      0.89      0.87     12500

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weigh

In [22]:
# Identify the best-performing model
best_model = max([(rnn_acc, 'RNN'), (lstm_acc, 'LSTM'), (gru_acc, 'GRU')])
print("Best Model:", best_model[1], "with Accuracy:", best_model[0])

Best Model: GRU with Accuracy: 0.86312


In [27]:
!git clone https://github.com/saroj-shah/NLP_Gen_AI/blob/main/Assignments

Cloning into 'Assignments'...
fatal: repository 'https://github.com/saroj-shah/NLP_Gen_AI/blob/main/Assignments/' not found


In [33]:
!git add nlp_assignment.ipynb

fatal: not a git repository (or any of the parent directories): .git
